<h1> Segmenting and Clustering Neighborhoods in Toronto </h1>

In this notebook, I will explore, segment, and cluster the neighborhoods in the city of Toronto

<h2>Importing libraries</h2>

Downloading required dependencies.

In [7]:
import numpy as np 
import pandas as pd 
import json 

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#! pip install folium==0.5.0 
import folium 

#!conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup 

print('Libraries imported.')

     |████████████████████████████████| 79 kB 6.7 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=5ead452f6de67d79220992e6208fdf91cdbc5b0e31ea6085901947d371e13541
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


<h1>PART 1</h1>

<h2>Scraping data from Wikipedia and creating a dataframe</h2>

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood. This data is scraped from the following Wikipedia page, using the BeautifulSoup package. 
Only cells that have an assigned borough are processed. Cells with a borough that is "Not assigned" are ignored. If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.


In [8]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [9]:
soup = BeautifulSoup(data, 'html.parser')

In [10]:
Postcode_list = []
Borough_list = []
Neighborhood_list = []

In [11]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        Postcode_list.append(cells[0].text.rstrip('\n'))
        Borough_list.append(cells[1].text.rstrip('\n'))
        Neighborhood_list.append(cells[2].text.rstrip('\n'))

In [18]:
toronto_df = pd.DataFrame({"Postcode": Postcode_list,
                           "Borough": Borough_list,
                           "Neighborhood": Neighborhood_list})

toronto_df.head()


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h2>Droping unassigned boroughs</h2>

In [19]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2>Replacing unassigned neighborhoods by borough</h2>

In [20]:
for index, row in toronto_df_dropna.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_dropna.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [21]:
toronto_df_dropna.shape

(103, 3)

<h1>PART 2</h1>

<h2>Reading Coordinates from online dataframe</h2>

In [22]:
coord_data = pd.read_csv('http://cocl.us/Geospatial_data')
coord_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
coord_data.rename(columns={"Postal Code": "Postcode"}, inplace=True)
coord_data.head()
toronto_df_new = toronto_df_dropna.merge(coord_data, on="Postcode", how="left")
toronto_df_new.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [65]:
toronto_df_new.shape

(103, 5)

<h1>PART 3</h1>

<h2>Narrowing down data on borough-names containing "Toronto"</h2>

In [27]:

# filter borough names that contain the word Toronto
boroughs = list(toronto_df_new.Borough.unique())
toronto_boroughs = []

for x in boroughs:
    if "toronto" in x.lower():
        toronto_boroughs.append(x)
        
toronto_boroughs

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [29]:
neighborhoods = toronto_df_new[toronto_df_new['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print(neighborhoods.shape)
neighborhoods.head()

(39, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Check number of boroughs and neighborhoods:

In [30]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 39 neighborhoods.


<h2>Using geopy library to get the latitude and longitude values of Toronto</h2> 

In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h2>Creating a map of Toronto with neighborhoods superimposed on top </h2> 

In [33]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h2>Defining Foursquare Credentials and Version</h2>

In [34]:
CLIENT_ID = 'HHXEV0A4IQXTLMZGJAIO3H0RVGVZIBR1EMCLFMIDDAV5ONGS' # your Foursquare ID
CLIENT_SECRET = 'IHR20VKQBFQUB453YFTMA1F11ITMHIQIP44S0NER0VAYLGHQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HHXEV0A4IQXTLMZGJAIO3H0RVGVZIBR1EMCLFMIDDAV5ONGS
CLIENT_SECRET:IHR20VKQBFQUB453YFTMA1F11ITMHIQIP44S0NER0VAYLGHQ


<h2>Getting the top 100 venues in Toronto within a radius of 500 meters</h2>

In [36]:
LIMIT = 100
radius = 500

venues_list=[]

for latitude, longitude, postcode, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Postcode'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues_list.append((
            latitude, 
            longitude,
            postcode,
            borough, 
            neighborhood, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


<h2>Structuring the json into a pandas dataframe</h2>

In [37]:
venues_df = pd.DataFrame(venues_list)
venues_df.columns = ['Latitude', 'Longitude', 'Postcode', 'Borough', 'Neighborhood', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [38]:
venues_df.head()

,Latitude,Longitude,Postcode,Borough,Neighborhood,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,43.65426,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,43.653447,-79.362017,Bakery
1,43.65426,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,43.65426,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront",Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,43.65426,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront",Impact Kitchen,43.656369,-79.356980,Restaurant
4,43.65426,-79.360636,M5A,Downtown Toronto,"Regent Park, Harbourfront",Body Blitz Spa East,43.654735,-79.359874,Spa


In [39]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 236 uniques categories.


In [40]:
venues_df.groupby('Neighborhood').count()

,Latitude,Longitude,Postcode,Borough,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,,,
Berczy Park,55,55,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16,16,16
Central Bay Street,68,68,68,68,68,68,68,68
Christie,16,16,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100,100
Davisville,33,33,33,33,33,33,33,33


Let's find out how many unique categories can be curated from all the returned venues:

In [41]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Restaurant',
       'Spa', 'Park', 'Pub', 'Breakfast Spot', 'Historic Site',
       'Gym / Fitness Center', 'Farmers Market', 'Chocolate Shop',
       'Performing Arts Venue', 'Theater', 'French Restaurant',
       'Dessert Shop', 'Mexican Restaurant', 'Yoga Studio', 'Café',
       'Event Space', 'Shoe Store', 'Art Gallery', 'Electronics Store',
       'Brewery', 'Bank', 'Beer Store', 'Hotel', 'Antique Shop',
       'Italian Restaurant', 'Portuguese Restaurant', 'Beer Bar',
       'Creperie', 'Sushi Restaurant', 'Hobby Shop', 'Diner',
       'Fried Chicken Joint', 'Chinese Restaurant', 'Smoothie Shop',
       'Sandwich Place', 'Gym', 'Bar', 'College Auditorium',
       'College Cafeteria', 'Music Venue', 'Clothing Store', 'Comic Shop',
       'Plaza', 'Burrito Place', 'Pizza Place', 'Ramen Restaurant'],
      dtype=object)

<h2>Analyzing Each Neighborhood</h2>

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

In [45]:
# adding neighborhood column back to dataframe
toronto_onehot['Postcode'] = venues_df['Postcode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

#moving new columns to the left
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]


In [46]:
toronto_onehot.head()

,Postcode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(1624, 239)

<h2>Grouping rows by neighborhood</h2>

In [48]:
toronto_grouped = toronto_onehot.groupby(["Postcode", "Borough", "Neighborhoods"]).mean().reset_index()

toronto_grouped.head()

,Postcode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.000000,0.0,0.023256
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.027027,0.0,0.027027
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [49]:
toronto_grouped.shape

(39, 239)

<h2>Showing each neighborhood along with the top 10 most common venues</h2>

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode', 'Borough', 'Neighborhoods']
count = []
for ind in np.arange(num_top_venues):
    try:
        count.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        count.append('{}th Most Common Venue'.format(ind+1))
columns = columns+count

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,Postcode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Lounge,Spa,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Gym,Sushi Restaurant,Sandwich Place,Liquor Store,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Café,Gastropub,American Restaurant,Brewery,Bakery,Yoga Studio,Bar,Ice Cream Shop,Fish Market
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
5,M4P,Central Toronto,Davisville North,Park,Hotel,Breakfast Spot,Sandwich Place,Food & Drink Shop,Dog Run,Dance Studio,Department Store,Gym / Fitness Center,Cosmetics Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Toy / Game Store,Greek Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pizza Place,Light Rail Station,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant,Fried Chicken Joint


<h2>Clustering Neighborhoods</h2>

In [51]:
# setting number of clusters to 5
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Neighborhoods','Postcode', 'Borough'], 1)

# running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 2, 0, 0, 0, 3, 0], dtype=int32)

In [52]:
# adding clustering labels
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [262]:
neighborhoods_venues_sorted

,Cluster Labels,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M4E,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Lounge,Spa,Indian Restaurant
2,1,M4L,Park,Gym,Sushi Restaurant,Sandwich Place,Liquor Store,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop
3,1,M4M,Coffee Shop,Café,Gastropub,American Restaurant,Brewery,Bakery,Yoga Studio,Bar,Ice Cream Shop,Fish Market
4,2,M4N,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
5,1,M4P,Park,Hotel,Breakfast Spot,Sandwich Place,Food & Drink Shop,Dog Run,Dance Studio,Department Store,Gym / Fitness Center,Cosmetics Shop
6,1,M4R,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
7,1,M4S,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Toy / Game Store,Greek Restaurant
8,4,M4T,Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,1,M4V,Coffee Shop,Pizza Place,Light Rail Station,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant,Fried Chicken Joint


<h2>Creating a new dataframe including all relevant info</h2>

In [53]:
toronto_merged = neighborhoods.copy()


In [55]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index("Postcode"), on="Postcode")

print(toronto_merged.shape)


(39, 16)


In [56]:
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Electronics Store,Spa,Beer Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


<h2>Visualizing the clusters</h2>

In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examining the clusters</h2>

<h3>Cluster 1</h3>

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Electronics Store,Spa,Beer Store
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop
2,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,St. James Town,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant
5,Berczy Park,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Bakery,Sandwich Place,Department Store
6,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Department Store,Burger Joint,Japanese Restaurant,Thai Restaurant
7,Christie,0,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Nightclub,Baby Store,Restaurant,Athletics & Sports,Coffee Shop
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Hotel,Gym,Restaurant,Clothing Store,Thai Restaurant,Bar,Bookstore,Burrito Place
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Park,Middle Eastern Restaurant,Bank,Bar,Café,Supermarket,Music Venue,Brewery
10,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Fried Chicken Joint,Brewery,Scenic Lookout,Bar,Music Venue


<h3>Cluster 2</h3>

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,1,Music Venue,Garden,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


<h3>Cluster 3</h3>

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,2,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
33,Rosedale,2,Park,Trail,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


<h3>Cluster 4</h3>

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",3,Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


<h3>Cluster 5</h3>

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,4,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
